<a href="https://colab.research.google.com/github/AdityaDevadiga/ETL/blob/main/task5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install boto3

In [ ]:
pip install snowflake-connector-python

In [ ]:
import boto3
import chardet
import pandas as pd
from io import StringIO
import snowflake.connector
import os

# Connect to S3
s3 = boto3.client('s3',
                  aws_access_key_id='****************************',
                  aws_secret_access_key='*******************************')

# Retrieve data from S3
bucket_name = 'your_bucket_name'
object_key = 'your_object_name'

s3_response = s3.get_object(Bucket=bucket_name, Key=object_key)
s3_data = s3_response['Body'].read()

# Detect the encoding
result = chardet.detect(s3_data)
actual_encoding = result['encoding']

# Decode using the detected encoding
decoded_data = s3_data.decode(actual_encoding)

# Read CSV data into a DataFrame
df = pd.read_csv(StringIO(decoded_data))

# Count the total number of rows in the dataset
num_of_rows = df.shape[0]

# Count the number of rows with null or empty cells
rows_with_null = df[df.isnull().any(axis=1)]
num_rows_with_null = len(rows_with_null)

# Count columns with null or empty cells
columns_with_null = df.columns[df.isnull().any()]
num_columns_with_null = len(columns_with_null)

# Count the number of rows with all columns having values
complete_rows = df.dropna()
num_complete_rows = len(complete_rows)

# Add a new column that counts null values in each row
df['Null_Count_row'] = df.isnull().sum(axis=1)

# Count the number of columns with null values
num_columns_with_null = df.isnull().any().sum()

# Print results
print("Total number of records in dataset:", num_of_rows)
print("Number of rows with null or empty cells:", num_rows_with_null)
print("Number of columns with null or empty cells:", num_columns_with_null)
print("Number of rows with all columns having values:", num_complete_rows)

# Create a Snowflake connection
conn = snowflake.connector.connect(
    user='user_name',
    password='password',
    account='account_name',
    warehouse='warehouse_name',
    database='result',
    schema='your_schema'
)

# Create a cursor
cur = conn.cursor()

# Create a table in Snowflake to store the results
create_table_sql = '''
CREATE OR REPLACE TABLE result_table (
    total_records INT,
    rows_with_null INT,
    columns_with_null INT,
    complete_rows INT
)
'''

cur.execute(create_table_sql)

# Insert results into Snowflake table
insert_sql = '''
INSERT INTO result_table (total_records, rows_with_null, columns_with_null, complete_rows)
VALUES (%s, %s, %s, %s)
'''

cur.execute(insert_sql, (num_of_rows, num_rows_with_null, num_columns_with_null, num_complete_rows))

# Commit the transaction
conn.commit()

# Close the Snowflake connection
cur.close()
conn.close()


Total number of records in dataset: 995
Number of rows with null or empty cells: 441
Number of columns with null or empty cells: 18
Number of rows with all columns having values: 554
